In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import shutil
import json
import itertools
import seaborn as sns
from PIL import Image
from collections import Counter
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
work_dir = '../input/cassava-leaf-disease-classification/'
os.listdir(work_dir)
train_path = '/kaggle/input/cassava-leaf-disease-classification/train_images/'

In [ ]:
data = pd.read_csv(work_dir + 'train.csv')
# data.head()
print(Counter(data['label']))

In [ ]:
data['label'].hist()

In [ ]:
f = open(work_dir + 'label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k, v in real_labels.items()}

data['class_name'] = data.label.map(real_labels)
print(data.head())

In [ ]:
data['class_name'].unique()

In [ ]:
def showImages(images):
    random_images = [np.random.choice(images) for i in range(16)]
    
    #Adjust size of image
    plt.figure(figsize = (16, 12))
    
    
    for i in range(5):
        plt.subplot(4, 4, i+1)
        img = plt.imread(train_path + random_images[i])
        plt.imshow(img, cmap = 'gray')
        plt.axis('off')
        
    plt.tight_layout()

In [ ]:
mask = data['label'] == 4
classHealthy = data[mask]

In [ ]:
showImages(classHealthy['image_id'])

In [ ]:
classCBB = data[data['label'] == 0]
classCBSD = data[data['label'] == 1]
classCMD = data[data['label'] == 3]
classCGM = data[data['label'] == 2]

In [ ]:
class0 = classCBB.sample(frac = 0.99)
class1 = classCBSD.sample(frac = 0.9)
class2 = classCGM.sample(frac = 0.9)
class3 = classCMD.sample(frac = 0.9)
class4 = classHealthy.sample(frac = 0.9)

frames = [class0, class1, class2, class3, class4]
finalData = pd.concat(frames)
# finalData.tail()
# finalData.head()
data = finalData['image_id']
label = finalData['label']

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(finalData, test_size = 0.05, random_state = 21, stratify = finalData['class_name'])

In [ ]:
train.head()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

img_size = 227
size = (img_size, img_size)
n_class = 5

datapen = ImageDataGenerator(preprocessing_function = None,
                            rotation_range = 60,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip = True,
                            vertical_flip = True,
                            fill_mode = 'nearest'
                            )

train_set = datapen.flow_from_dataframe(train,
                                       directory = train_path,
                                       seed = 7,
                                       x_col = 'image_id',
                                       y_col = 'class_name',
                                       target_size = size,
                                       class_mode = 'categorical',
                                       interpolation = 'nearest',
                                       shuffle = True,
                                       batch_size = 32)

val_set = datapen.flow_from_dataframe(val,
                                       directory = train_path,
                                       seed = 7,
                                       x_col = 'image_id',
                                       y_col = 'class_name',
                                       target_size = size,
                                       class_mode = 'categorical',
                                       interpolation = 'nearest',
                                       shuffle = True,
                                       batch_size = 32)

In [ ]:
def create_model():
    model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
        
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
        
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
        
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
        
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
        
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(5, activation='softmax')
    ])
    
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
epoch = 30
step_size_train = train_set.n//train_set.batch_size
step_size_valid = val_set.n//val_set.batch_size

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
#       keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      tf.keras.metrics.CategoricalAccuracy(name = 'categorical_accuracy')
]

In [ ]:
 def model_fit():
    leaf_model = create_model()
    
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False,
                                                   label_smoothing = 0.001,
                                                   name = 'categorical_crossentropy')
    
    leaf_model.compile(optimizer = Adam(learning_rate = 2e-5),
                      loss = loss,
                      metrics = METRICS)
    
    es = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 5, restore_best_weights = True, verbose = 1)
    
    
    checkpoint_cb = ModelCheckpoint('Cassava_model',
                                   save_best_only = True,
                                   monitor = 'val_loss',
                                   mode = 'min')
    
    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                                 factor = 0.3,
                                 patience = 3,
                                 min_lr = 1e-6,
                                 mode = 'min',
                                 verbose = 1)
    
    history = leaf_model.fit(train_set,
                            validation_data = val_set,
                            epochs = epoch,
                            steps_per_epoch = step_size_train,
                            validation_steps = step_size_valid,
                            callbacks = [es, checkpoint_cb, reduce_lr])
    
    
    leaf_model.save('Cassava_model_alexnet.h5')
    
    return history

In [ ]:
leaf_model = create_model()
history = model_fit()

In [ ]:
tp1 = history.history['tp'][-1]
print(tp1)
fp1 = history.history['fp'][-1]
print(fp1)
tn1 = history.history['tn'][-1]
print(tn1)
fn1 = history.history['fn'][-1]
print(fn1)

pre = history.history['precision'][-1]
print(pre)
rec = history.history['recall'][-1]
print(rec)
acc = history.history['categorical_accuracy'][-1]
print(acc)
val_acc = history.history['val_categorical_accuracy'][-1]
print(val_acc)
f1_score=2*pre*rec/(pre+rec)
print(f1_score)


In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epoch)

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label = 'Training accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training loss')
plt.plot(epochs_range, val_loss, label = 'Validation loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()